# How to Build a Recommendation System for Purchase Data (Step-by-Step)
* Description: A documentation on building collaborative filtering models for recommending products to customers
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Author: Moorissa Tjokro

## Problem statement
In this data challenge, we are building collaborative filtering models for recommending product items. The steps below aim to recommend users their top 10 items to place into their basket. The final output will be a csv file in the `output` folder, and a function that searches for a recommendation list based on a speficied user:
* Input: user - customer ID
* Returns: ranked list of items (product IDs), that the user is most likely to want to put in his/her (empty) "basket"

## 1. Import modules
* `pandas` and `numpy` for data manipulation
* `turicreate` for performing model selection and evaluation
* `sklearn` for splitting the data into train and test set

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
import data_layer as data_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load data
Two datasets are used in this exercise, which can be found in `data` folder: 
* `recommend_1.csv` consisting of a list of 1000 customer IDs to recommend as output
* `trx_data.csv` consisting of user transactions

The format is as follows.

In [4]:
customers = pd.read_csv('data/recommend_1.csv')
transactions = pd.read_csv('data/trx_data.csv')

In [5]:
print(customers.shape)
customers.head()

(1000, 1)


customerId
0        1553
1       20400
2       19750
3        6334
4       27773

In [6]:
print(transactions.shape)
transactions.head()

(62483, 2)


customerId                        products
0           0                              20
1           1  2|2|23|68|68|111|29|86|107|152
2           2       111|107|29|11|11|11|33|23
3           3                         164|227
4           5                             2|2

## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [7]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

customerId     0    1     2     3     4      5     6     7      8      9
0           0  20.0  NaN   NaN   NaN   NaN    NaN   NaN   NaN    NaN    NaN
1           1   2.0  2.0  23.0  68.0  68.0  111.0  29.0  86.0  107.0  152.0

In [8]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

customerId  productId  purchase_count
0           0       20.0               1
1           1        2.0               2
2           1       23.0               1
3           1       29.0               1
4           1       68.0               2
5           1       86.0               1
6           1      107.0               1
7           1      111.0               1
8           1      152.0               1

### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [9]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.23 minutes


In [10]:
print(data.shape)
data.head()

(133585, 3)


customerId  productId  purchase_count
0           0          1               2
1           0         13               1
2           0         19               3
3           0         20               1
4           0         31               2

### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [11]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [12]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [13]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId   0    1    2    3    4    5    6    7    8    9    ...  290  291  \
customerId                                                    ...             
0           NaN  2.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
1           NaN  NaN  6.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
3           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

productId   292  293  294  295  296  297  298  299  
customerId                                          
0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1           NaN  1.0  NaN  NaN  1.0  NaN  NaN  NaN  
2           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 300 columns]

In [14]:
(df_matrix.shape)

(24429, 300)

In [15]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId   0    1         2    3    4    5    6    7    8    9    ...  290  \
customerId                                                         ...        
0           NaN  0.1       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
1           NaN  NaN  0.166667  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
2           NaN  NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
3           NaN  NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
4           NaN  NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   

productId   291  292  293  294  295  296  297  298  299  
customerId                                               
0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1           NaN  NaN  0.0  NaN  NaN  0.0  NaN  NaN  NaN  
2           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 300 columns]

In [16]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


customerId productId  scaled_purchase_freq
9            9         0              0.133333
25          25         0              0.133333
32          33         0              0.133333
35          36         0              0.133333
43          44         0              0.133333

#### Define a function for normalizing data

In [17]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [18]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [19]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [20]:
train_data

Columns:
	customerId	int
	productId	int
	purchase_count	int

Rows: 106868

Data:
+------------+-----------+----------------+
| customerId | productId | purchase_count |
+------------+-----------+----------------+
|    2781    |     16    |       1        |
|   25682    |    105    |       1        |
|   12725    |     1     |       1        |
|    8936    |    114    |       1        |
|   17307    |     98    |       3        |
|   12929    |    208    |       2        |
|    3543    |    269    |       1        |
|   18594    |     7     |       1        |
|    6897    |     1     |       2        |
|    7118    |    231    |       1        |
+------------+-----------+----------------+
[106868 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [21]:
test_data

Columns:
	customerId	int
	productId	int
	purchase_count	int

Rows: 26717

Data:
+------------+-----------+----------------+
| customerId | productId | purchase_count |
+------------+-----------+----------------+
|    9160    |     78    |       1        |
|    2946    |     19    |       1        |
|    2545    |    156    |       1        |
|   21256    |    224    |       1        |
|    1709    |     92    |       2        |
|    6016    |     23    |       1        |
|    5402    |     50    |       1        |
|    2222    |     89    |       1        |
|   22731    |     43    |       2        |
|   16666    |    101    |       1        |
+------------+-----------+----------------+
[26717 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

#### Define a `split_data` function for splitting data to training and test set

In [22]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [23]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [24]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [59]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Recsys training: model = popularity

Preparing data set.

Data has 106868 observations with 23244 users and 300 items.

Data prepared in: 0.230467s

106868 observations to process; with 300 unique items.

In [60]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 37387.4

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.111111111111111  |  1   |
|    1553    |     37    | 3.0988142292490117 |  2   |
|    1553    |     34    | 3.030888030888031  |  3   |
|    1553    |    132    | 3.015873015873016  |  4   |
|    1553    |     0     | 2.959847036328872  |  5   |
|    1553    |     3     | 2.853249475890985  |  6   |
|    1553    |     27    | 2.769230769230769  |  7   |
|    1553    |    110    | 2.7349397590361444 |  8   |
|    1553    |     32    | 2.650717703349282  |  9   |
|    1553    |     82    | 2.609865470852018  |  10  |
|   20400    |    248    | 3.111111111111111  |  1   |
|   20400    |     37    | 3.0988142292490117 |  2   |
|   20400    |     34    | 3.030888030888031  |  3   |
|   20400    |    132    | 3.015873015873016  |  4   |
|   20400    |     0     | 2.959847036328872  |  5   |
|   20400 

#### Define a `model` function for model selection

In [62]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [64]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [65]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = popularity

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23321 users and 300 items.

Data prepared in: 0.212271s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 41535.1

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     17    |  1.0  |  1   |
|    1553    |     21    |  1.0  |  2   |
|    1553    |     13    |  1.0  |  3   |
|    1553    |     1     |  1.0  |  4   |
|    1553    |     76    |  1.0  |  5   |
|    1553    |     47    |  1.0  |  6   |
|    1553    |    101    |  1.0  |  7   |
|    1553    |     25    |  1.0  |  8   |
|    1553    |    174    |  1.0  |  9   |
|    1553    |    186    |  1.0  |  10  |
|   20400    |     17    |  1.0  |  1   |
|   20400    |     21    |  1.0  |  2   |
|   20400    |     13    |  1.0  |  3   |
|   20400    |     1     |  1.0  |  4   |
|   20400    |     76    |  1.0  |  5   |
|   20400    |     47    |  1.0  |  6   |
|   20400    |    101    |  1.0  |  7   |
|   20400    |     25    |  1.0  |  8   |
|   20400    |    174    |  1.0  |  9   |
|   20400    |    186    |  1.0  |  10  |
|   19750    |     17    |  1.0  |

#### Using normalized purchase count

In [67]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = popularity

Preparing data set.

Data has 106868 observations with 23336 users and 300 items.

Data prepared in: 0.202731s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 29169

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7931034482758621 |  1   |
|    1553    |    247    |  0.3358208955223881 |  2   |
|    1553    |    230    | 0.31532846715328416 |  3   |
|    1553    |    125    | 0.26029411764705845 |  4   |
|    1553    |    294    |  0.2573643410852709 |  5   |
|    1553    |    248    | 0.25555555555555554 |  6   |
|    1553    |    276    | 0.24807692307692308 |  7   |
|    1553    |    165    | 0.23036649214659685 |  8   |
|    1553    |    155    | 0.22807017543859645 |  9   |
|    1553    |     72    |  0.2268993839835729 |  10  |
|   20400    |    226    |  0.7931034482758621 |  1   |
|   20400    |    247    |  0.3358208955223881 |  2   |
|   20400    |    230    | 0.31532846715328416 |  3   |
|   20400    |    125    | 0.26029411764705845 |  4   |
|   20400    |    294    |  0.2573643410852709 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [68]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
248    3.111111
37     3.098814
34     3.030888
132    3.015873
0      2.959847
3      2.853249
27     2.769231
110    2.734940
32     2.650718
82     2.609865
10     2.608955
230    2.584615
129    2.584337
226    2.557823
245    2.554455
58     2.489362
68     2.432203
54     2.405109
18     2.400000
91     2.393162
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [78]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Preparing data set.

Data has 106868 observations with 23244 users and 300 items.

Data prepared in: 0.251345s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.073006s

recommendations finished on 1000/1000 queries. users per second: 53847.4

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     |  0.1110002875328064  |  1   |
|    1553    |     35    | 0.08345692157745362  |  2   |
|    1553    |     1     | 0.07202702760696411  |  3   |
|    1553    |     5     | 0.06954102516174317  |  4   |
|    1553    |     17    | 0.062017965316772464 |  5   |
|    1553    |     61    | 0.06159121990203857  |  6   |
|    1553    |     21    | 0.051773107051849364 |  7   |
|    1553    |     41    | 0.048556816577911374 |  8   |
|    1553    |     47    | 0.04801986217498779  |  9   |
|    1553    |     82    | 0.04561170339584351  |  10  |
|   20400    |     1     | 0.049841105937957764 |  1   |
|   20400    |    182    |  0.0481337308883667  |  2   |
|   20400    |     13    |  0.0420384407043457  |  3   |
|   20400    |     56    | 0.04202163219451904  |  4   |
|   20400    |    215    | 0.04

#### Using purchase dummy

In [79]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23321 users and 300 items.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.076872s

recommendations finished on 1000/1000 queries. users per second: 13894.5

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11069482564926147  |  1   |
|    1553    |     1     | 0.07363476355870564  |  2   |
|    1553    |     5     | 0.06431001424789429  |  3   |
|    1553    |     21    | 0.06199751297632853  |  4   |
|    1553    |     17    |  0.0569839080174764  |  5   |
|    1553    |     61    | 0.05539397398630778  |  6   |
|    1553    |     29    | 0.05539158980051676  |  7   |
|    1553    |    105    | 0.053917884826660156 |  8   |
|    1553    |     15    | 0.05295040210088094  |  9   |
|    1553    |    143    | 0.051859756310780845 |  10  |
|   20400    |     17    |         0.0          |  1   |
|   20400    |     21    |         0.0          |  2   |
|   20400    |     13    |         0.0          |  3   |
|   20400    |     1     |         0.0          |  4   |
|   20400    |     76    |     

#### Using normalized purchase count

In [80]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Preparing data set.

Data has 106868 observations with 23336 users and 300 items.

Data prepared in: 0.232637s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.079409s

recommendations finished on 1000/1000 queries. users per second: 40864.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     18    |  0.0  |  1   |
|    1553    |     2     |  0.0  |  2   |
|    1553    |    225    |  0.0  |  3   |
|    1553    |    174    |  0.0  |  4   |
|    1553    |     9     |  0.0  |  5   |
|    1553    |    147    |  0.0  |  6   |
|    1553    |     78    |  0.0  |  7   |
|    1553    |     4     |  0.0  |  8   |
|    1553    |    270    |  0.0  |  9   |
|    1553    |    267    |  0.0  |  10  |
|   20400    |     18    |  0.0  |  1   |
|   20400    |     2     |  0.0  |  2   |
|   20400    |    225    |  0.0  |  3   |
|   20400    |    174    |  0.0  |  4   |
|   20400    |     9     |  0.0  |  5   |
|   20400    |    147    |  0.0  |  6   |
|   20400    |     78    |  0.0  |  7   |
|   20400    |     4     |  0.0  |  8   |
|   20400    |    270    |  0.0  |  9   |
|   20400    |    267    |  0.0  |  10  |
|   19750    |     18    |  0.0  |

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [85]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Preparing data set.

Data has 106868 observations with 23244 users and 300 items.

Data prepared in: 0.215635s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.071802s

recommendations finished on 1000/1000 queries. users per second: 41606

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     |  0.1110002875328064  |  1   |
|    1553    |     35    | 0.08345692157745362  |  2   |
|    1553    |     1     | 0.07202702760696411  |  3   |
|    1553    |     5     | 0.06954102516174317  |  4   |
|    1553    |     17    | 0.062017965316772464 |  5   |
|    1553    |     61    | 0.06159121990203857  |  6   |
|    1553    |     21    | 0.051773107051849364 |  7   |
|    1553    |     41    | 0.048556816577911374 |  8   |
|    1553    |     47    | 0.04801986217498779  |  9   |
|    1553    |     82    | 0.04561170339584351  |  10  |
|   20400    |     1     | 0.049841105937957764 |  1   |
|   20400    |    182    |  0.0481337308883667  |  2   |
|   20400    |     13    |  0.0420384407043457  |  3   |
|   20400    |     56    | 0.04202163219451904  |  4   |
|   20400    |    215    | 0.04

#### Using purchase dummy

In [82]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23321 users and 300 items.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.09107s

recommendations finished on 1000/1000 queries. users per second: 25631.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11069482564926147  |  1   |
|    1553    |     1     | 0.07363476355870564  |  2   |
|    1553    |     5     | 0.06431001424789429  |  3   |
|    1553    |     21    | 0.06199751297632853  |  4   |
|    1553    |     17    |  0.0569839080174764  |  5   |
|    1553    |     61    | 0.05539397398630778  |  6   |
|    1553    |     29    | 0.05539158980051676  |  7   |
|    1553    |    105    | 0.053917884826660156 |  8   |
|    1553    |     15    | 0.05295040210088094  |  9   |
|    1553    |    143    | 0.051859756310780845 |  10  |
|   20400    |     17    |         0.0          |  1   |
|   20400    |     21    |         0.0          |  2   |
|   20400    |     13    |         0.0          |  3   |
|   20400    |     1     |         0.0          |  4   |
|   20400    |     76    |     

#### Using normalized purchase count

In [83]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Recsys training: model = item_similarity

Preparing data set.

Data has 106868 observations with 23336 users and 300 items.

Data prepared in: 0.208176s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.067378s

recommendations finished on 1000/1000 queries. users per second: 14962.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     18    |  0.0  |  1   |
|    1553    |     2     |  0.0  |  2   |
|    1553    |    225    |  0.0  |  3   |
|    1553    |    174    |  0.0  |  4   |
|    1553    |     9     |  0.0  |  5   |
|    1553    |    147    |  0.0  |  6   |
|    1553    |     78    |  0.0  |  7   |
|    1553    |     4     |  0.0  |  8   |
|    1553    |    270    |  0.0  |  9   |
|    1553    |    267    |  0.0  |  10  |
|   20400    |     18    |  0.0  |  1   |
|   20400    |     2     |  0.0  |  2   |
|   20400    |    225    |  0.0  |  3   |
|   20400    |    174    |  0.0  |  4   |
|   20400    |     9     |  0.0  |  5   |
|   20400    |    147    |  0.0  |  6   |
|   20400    |     78    |  0.0  |  7   |
|   20400    |     4     |  0.0  |  8   |
|   20400    |    270    |  0.0  |  9   |
|   20400    |    267    |  0.0  |  10  |
|   19750    |     18    |  0.0  |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [88]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [89]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13890 queries. users per second: 43512.3

recommendations finished on 2000/13890 queries. users per second: 54665.7

recommendations finished on 3000/13890 queries. users per second: 58828.1

recommendations finished on 4000/13890 queries. users per second: 59864.1

...

recommendations finished on 10000/13890 queries. users per second: 40582.4

recommendations finished on 11000/13890 queries. users per second: 40487.5

recommendations finished on 12000/13890 queries. users per second: 40891.7

recommendations finished on 13000/13890 queries. users per second: 41214.9


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.06357091432685413  | 0.035144393373017405 |
|   2    |  0.0624910007199426  | 0.07222392181915947  |
|   3    | 0.051091912646988306 | 0.08727520602811081  |
|   4    | 0.043178545716342755 |  0.0968562746107815  |
|   5    | 0.03832973362131065  | 0.10616781100655155  |
|   6    | 0.034569234461243395 | 0.11430058895574437  |
|   7    | 0.03175974493469068  | 0.12174051064512559  |
|   8    | 0.030039596832253546 | 0.13105989027280296  |
|   9    | 0.028413726901847833 | 0.13911332731652126  |
|   10   | 0.027041036717062584 | 0.14659112729432097  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.9231102838192284

Per User RMSE (best)
+------------+---------------------+-------+
| customerId |         rmse

#### Models on purchase dummy

In [91]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13921 queries. users per second: 50289.2

recommendations finished on 2000/13921 queries. users per second: 52588.7

recommendations finished on 3000/13921 queries. users per second: 58364.6

recommendations finished on 4000/13921 queries. users per second: 55436.2

...

recommendations finished on 10000/13921 queries. users per second: 45889.5

recommendations finished on 11000/13921 queries. users per second: 45760.7

recommendations finished on 12000/13921 queries. users per second: 45468.3

recommendations finished on 13000/13921 queries. users per second: 45588


Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |    mean_precision    |     mean_recall     |
+--------+----------------------+---------------------+
|   1    | 0.054881114862438046 | 0.03059300533530103 |
|   2    | 0.054737447022484356 | 0.06055703487263677 |
|   3    | 0.04575820702535712  | 0.07499666486309599 |
|   4    | 0.03778464190790865  | 0.08163578704079291 |
|   5    | 0.03364700811723305  | 0.09059373381031925 |
|   6    | 0.03131958910997744  | 0.09999963953352173 |
|   7    | 0.029605837019097744 | 0.10925854333753758 |
|   8    | 0.028140938150995002 | 0.11805506113304967 |
|   9    | 0.026953683084708142 |  0.1265609526547087 |
|   10   | 0.025716543351770624 | 0.13383989516818834 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9697745320187097

Per User RMSE (best)
+------------+--------------------+-------+
| customerId |        rmse        | count 

#### Models on normalized purchase frequency

In [92]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13887 queries. users per second: 28418

recommendations finished on 2000/13887 queries. users per second: 36583.8

recommendations finished on 3000/13887 queries. users per second: 40400.2

recommendations finished on 4000/13887 queries. users per second: 44710.7

...

recommendations finished on 10000/13887 queries. users per second: 47748.2

recommendations finished on 11000/13887 queries. users per second: 47962.5

recommendations finished on 12000/13887 queries. users per second: 48750.6

recommendations finished on 13000/13887 queries. users per second: 49660.2


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.023475192626197055 | 0.012186231083083403 |
|   2    | 0.03208036292935828  | 0.03676344205668188  |
|   3    | 0.02993207076162342  | 0.04983143488812296  |
|   4    | 0.028983941816086736 | 0.06361982403649855  |
|   5    | 0.029783250522070984 |  0.082266120971843   |
|   6    | 0.028827920597201143 | 0.09634521387973909  |
|   7    | 0.026911088479461782 | 0.10388903948175872  |
|   8    | 0.024996399510333337 | 0.10985690825476863  |
|   9    | 0.023771232887672755 | 0.11705134384086605  |
|   10   | 0.02281270252754377  | 0.12477339403969312  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1622982668334026

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+-----------

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [120]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Recsys training: model = item_similarity

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.281178s

Training model from provided data.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.130605s

recommendations finished on 1000/1000 queries. users per second: 40159

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.772043646706475  |  1   |
|    1553    |    247    |  0.3375527426160338 |  2   |
|    1553    |    230    |  0.3270096631277173 |  3   |
|    1553    |    125    | 0.25667036101662893 |  4   |
|    1553    |    294    |  0.2527607361963191 |  5   |
|    1553    |    248    | 0.24968572854995727 |  6   |
|    1553    |    204    | 0.23603603603603604 |  7   |
|    1553    |    276    | 0.22853291395938757 |  8   |
|    1553    |     72    |  0.2259392835912923 |  9   |
|    1553    |    155    | 0.22318840579710147 |  10  |
|   20400    |    226    |  0.772222222222222  |  1   |
|   20400    |    247    |  0.3375527426160338 |  2   |
|   20400    |    230    | 0.32738095238095216 |  3   |
|   20400    |    125    | 0.25621119067513726 |  4   |
|   20400    |    294    |  0.2527607361963191 |

### 8.1. CSV output file

In [121]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


customerId  productId     score  rank
0        1553        226  0.772044     1
1        1553        247  0.337553     2
2        1553        230  0.327010     3
3        1553        125  0.256670     4
4        1553        294  0.252761     5

In [122]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [125]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [126]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 38980.3

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


recommendedProducts
customerId                                 
4           2|82|249|14|86|215|39|194|111|8
11          0|51|2|103|31|169|13|226|11|271
12          44|109|170|1|82|2|19|276|118|47
16          14|162|1|47|17|105|21|223|118|0
21              48|38|93|36|79|2|50|144|1|0

### 8.2. Customer recommendation function

In [127]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [128]:
customer_recomendation(4)

recommendedProducts    2|82|249|14|86|215|39|194|111|8
Name: 4, dtype: object

In [129]:
customer_recomendation(21)

recommendedProducts    48|38|93|36|79|2|50|144|1|0
Name: 21, dtype: object

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 